In [27]:
import tensorflow as tf
   

In [29]:
import numpy as np 
import pandas as pd 
import json 
import csv 
import random 

import pprint 
import tensorflow as tf 
from tensorflow.python.framework import ops 
from sklearn.model_selection import train_test_split 
from sklearn import preprocessing 

from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.utils import to_categorical 
from tensorflow.keras import regularizers 

# Enable eager execution (optional, if needed)
tf.compat.v1.enable_eager_execution()

# Reading the data 
data = pd.read_csv("news (1).csv") 
data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [30]:
data = data.drop(["Unnamed: 0"], axis=1) 
data.head(5) 

,title,text,label
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [31]:
# encoding the labels 
le = preprocessing.LabelEncoder() 
le.fit(data['label']) 
data['label'] = le.transform(data['label']) 


In [32]:
embedding_dim = 50
max_length = 54
trunc_type = 'post'
padding_type = 'post'
oov_tok = "<OOV>"
training_size = 3000
test_portion = .1



In [33]:
title = [] 
text = [] 
labels = [] 
for x in range(training_size): 
	title.append(data['title'][x]) 
	text.append(data['text'][x]) 
	labels.append(data['label'][x]) 


In [34]:
tokenizer1 = Tokenizer() 
tokenizer1.fit_on_texts(title) 
word_index1 = tokenizer1.word_index 
vocab_size1 = len(word_index1) 
sequences1 = tokenizer1.texts_to_sequences(title) 
padded1 = pad_sequences( 
	sequences1, padding=padding_type, truncating=trunc_type) 
split = int(test_portion * training_size) 
training_sequences1 = padded1[split:training_size] 
test_sequences1 = padded1[0:split] 
test_labels = labels[0:split] 
training_labels = labels[split:training_size] 


In [38]:
import numpy as np

embeddings_index = {} 
with open('glove.6B.50d.txt', encoding='utf-8') as f: 
    for line in f: 
        values = line.split() 
        word = values[0] 
        coefs = np.asarray(values[1:], dtype='float32') 
        embeddings_index[word] = coefs 

# Generating embeddings 
embeddings_matrix = np.zeros((vocab_size1 + 1, embedding_dim)) 
for word, i in word_index1.items(): 
    embedding_vector = embeddings_index.get(word) 
    if embedding_vector is not None: 
        embeddings_matrix[i] = embedding_vector


In [47]:
model = tf.keras.Sequential([ 
	tf.keras.layers.Embedding(vocab_size1+1, embedding_dim, 
							input_length=max_length, weights=[ 
								embeddings_matrix], 
							trainable=False), 
	tf.keras.layers.Dropout(0.2), 
	tf.keras.layers.Conv1D(64, 5, activation='relu'), 
	tf.keras.layers.MaxPooling1D(pool_size=4), 
	tf.keras.layers.LSTM(64), 
	tf.keras.layers.Dense(1, activation='sigmoid') 
]) 
model.compile(loss='binary_crossentropy', 
			optimizer='adam', metrics=['accuracy']) 
model.summary() 


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │         377,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_2 (Conv1D)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_2 (MaxPooling1D)       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 377,600 (1.44 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 377,600 (1.44 MB)

In [40]:
num_epochs = 50

training_padded = np.array(training_sequences1) 
training_labels = np.array(training_labels) 
testing_padded = np.array(test_sequences1) 
testing_labels = np.array(test_labels) 

history = model.fit(training_padded, training_labels, 
					epochs=num_epochs, 
					validation_data=(testing_padded, 
									testing_labels), 
					verbose=2) 


Epoch 1/50
85/85 - 9s - 100ms/step - accuracy: 0.6241 - loss: 0.6450 - val_accuracy: 0.7100 - val_loss: 0.5663
Epoch 2/50
85/85 - 1s - 12ms/step - accuracy: 0.7037 - loss: 0.5756 - val_accuracy: 0.6967 - val_loss: 0.5432
Epoch 3/50
85/85 - 1s - 14ms/step - accuracy: 0.7285 - loss: 0.5342 - val_accuracy: 0.7067 - val_loss: 0.5383
Epoch 4/50
85/85 - 1s - 14ms/step - accuracy: 0.7570 - loss: 0.4949 - val_accuracy: 0.7200 - val_loss: 0.4953
Epoch 5/50
85/85 - 1s - 15ms/step - accuracy: 0.7889 - loss: 0.4545 - val_accuracy: 0.7067 - val_loss: 0.5174
Epoch 6/50
85/85 - 1s - 14ms/step - accuracy: 0.8137 - loss: 0.4056 - val_accuracy: 0.7200 - val_loss: 0.5125
Epoch 7/50
85/85 - 1s - 15ms/step - accuracy: 0.8326 - loss: 0.3704 - val_accuracy: 0.7367 - val_loss: 0.5075
Epoch 8/50
85/85 - 1s - 16ms/step - accuracy: 0.8607 - loss: 0.3242 - val_accuracy: 0.7500 - val_loss: 0.4918
Epoch 9/50
85/85 - 1s - 15ms/step - accuracy: 0.8874 - loss: 0.2815 - val_accuracy: 0.7433 - val_loss: 0.5394
Epoch 10/

In [41]:
# sample text to check if fake or not 
X = "Karry to go to France in gesture of sympathy"

# detection 
sequences = tokenizer1.texts_to_sequences([X])[0] 
sequences = pad_sequences([sequences], maxlen=54, 
						padding=padding_type, 
						truncating=trunc_type) 
if(model.predict(sequences, verbose=0)[0][0] >= 0.5): 
	print("This news is True") 
else: 
	print("This news is false") 


This news is True
